In [1]:
import numpy as np
f = open('input/BLOSUM62.txt', 'r')
score = [line.strip().split() for line in f]
alphabet = score[0]
blosum62 = np.array([i[1:] for i in score[1:]], dtype = 'int')
print alphabet
print blosum62

['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '-']
[[ 4  0 -2 -1 -2  0 -2 -1 -1 -1 -1 -2 -1 -1 -1  1  0  0 -3 -2 -5]
 [ 0  9 -3 -4 -2 -3 -3 -1 -3 -1 -1 -3 -3 -3 -3 -1 -1 -1 -2 -2 -5]
 [-2 -3  6  2 -3 -1 -1 -3 -1 -4 -3  1 -1  0 -2  0 -1 -3 -4 -3 -5]
 [-1 -4  2  5 -3 -2  0 -3  1 -3 -2  0 -1  2  0  0 -1 -2 -3 -2 -5]
 [-2 -2 -3 -3  6 -3 -1  0 -3  0  0 -3 -4 -3 -3 -2 -2 -1  1  3 -5]
 [ 0 -3 -1 -2 -3  6 -2 -4 -2 -4 -3  0 -2 -2 -2  0 -2 -3 -2 -3 -5]
 [-2 -3 -1  0 -1 -2  8 -3 -1 -3 -2  1 -2  0  0 -1 -2 -3 -2  2 -5]
 [-1 -1 -3 -3  0 -4 -3  4 -3  2  1 -3 -3 -3 -3 -2 -1  3 -3 -1 -5]
 [-1 -3 -1  1 -3 -2 -1 -3  5 -2 -1  0 -1  1  2  0 -1 -2 -3 -2 -5]
 [-1 -1 -4 -3  0 -4 -3  2 -2  4  2 -3 -3 -2 -2 -2 -1  1 -2 -1 -5]
 [-1 -1 -3 -2  0 -3 -2  1 -1  2  5 -2 -2  0 -1 -1 -1  1 -1 -1 -5]
 [-2 -3  1  0 -3  0  1 -3  0 -3 -2  6 -2  0  0  1  0 -3 -4 -2 -5]
 [-1 -3 -1 -1 -4 -2 -2 -3 -1 -3 -2 -2  7 -1 -2 -1 -1 -2 -4 -3 -5]
 [-1 -3  0  2 -3 -2  0 -3  1 -2  0  

In [2]:
blosum62[alphabet.index('A'), alphabet.index('W')]

-3

In [3]:
blosum62[alphabet.index('A'), alphabet.index('-')]

-5

In [4]:
def globalAlignment(x, y, score):
    """ Calculate global alignment value of sequences x and y using
        dynamic programming.  Return global alignment value. """
    
    D = np.zeros((len(x)+1, len(y)+1), dtype = 'int')
    
    for i in range(1, len(x) + 1):
        D[i,0] = D[i-1,0] + score[alphabet.index(x[i-1]), alphabet.index('-')]
        
    for j in range(1, len(y) + 1):
        D[0,j] = D[0,j-1] + score[alphabet.index('-'), alphabet.index(y[j-1])]
        
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            horz = D[i, j-1] + score[alphabet.index('-'), alphabet.index(y[j-1])]
            vert = D[i-1, j] + score[alphabet.index(x[i-1]), alphabet.index('-')]
            diag = D[i-1, j-1] + score[alphabet.index(x[i-1]), alphabet.index(y[j-1])]
            D[i,j] = max(horz, vert, diag)
    return D, D[-1][-1]

In [5]:
D, max_score = globalAlignment('PLEASANTLY','MEANLY', blosum62)
print (D)
print (max_score)

[[  0  -5 -10 -15 -20 -25 -30]
 [ -5  -2  -6 -11 -16 -21 -26]
 [-10  -3  -5  -7 -12 -12 -17]
 [-15  -8   2  -3  -7 -12 -14]
 [-20 -13  -3   6   1  -4  -9]
 [-25 -18  -8   1   7   2  -3]
 [-30 -23 -13  -4   2   6   1]
 [-35 -28 -18  -9   2   1   4]
 [-40 -33 -23 -14  -3   1  -1]
 [-45 -38 -28 -19  -8   1   0]
 [-50 -43 -33 -24 -13  -4   8]]
8


In [13]:
def traceback(D, x, y, score):
    """ Trace back from given cell in global-alignment matrix D """
    # get i, j for maximal cell
    i, j = len(x), len(y)
    #i, j = np.unravel_index(np.argmax(D), D.shape)
    alx, aly = [], []
    while (i,j) != (0,0):
    #while (i > 0 or j > 0) and D[i,j] != 0:
        diag, horz, vert = -float('inf'), -float('inf'), -float('inf')
        if i > 0 and j > 0:
            diag = D[i-1, j-1] + score[alphabet.index(x[i-1]), alphabet.index(y[j-1])]
        if i > 0:
            vert = D[i-1, j] + score[alphabet.index(x[i-1]), alphabet.index('-')]
        if j > 0:
            horz = D[i, j-1] + score[alphabet.index('-'), alphabet.index(y[j-1])]
        if diag >= vert and diag >= horz:
            alx.append(x[i-1]); aly.append(y[j-1])
            i -= 1; j -= 1 
        elif vert >= horz:
            alx.append(x[i-1]); aly.append('-')
            i -= 1
        else:
            alx.append('-'); aly.append(y[j-1])
            j -= 1
    alignment = map(lambda x: ''.join(x), [alx[::-1], aly[::-1]])
    return alignment

In [14]:
x = 'PLEASANTLY'
y = 'MEANLY'
D, max_score = globalAlignment(x,y, blosum62)
algn = traceback(D, x, y, blosum62)
print '\n'.join(algn)

PLEASANTLY
-ME--AN-LY


In [17]:
x, y = [i.strip() for i in open('input/rosalind_ba5e.txt', 'r')]
D, max_score = globalAlignment(x, y, blosum62)
algn = traceback(D, x, y, blosum62)
print max_score
print '\n'.join(algn)

11050
WGW-AEFMEQRNPKGGEKWHHRYGSY-HAMRRWEPANETNCDPKDHLNPRNINHWYGTMHKATVWVRELQQSNAASDCYIG--I--VGL-A-------IYEMVQFLHQCRYVIVLWASMMFLYAPARMPGQYHSN----RSDSRSDV---E-------C-----DCGTSPWCWAWGFPAYNLERWKPHDFHSYCHRRQRVS-----H-IEQYHK--F--K----YHLTHIKRTNEFHQNPHHLKTDNRFGARDFRQFLAFTQRHMTPTMWVEQDEKAN-HYRYCRLSINDNKITVVYTI--WLFVVEMQDLAQKGFPVRKYMHQWCTTFLQSAFYFVDFLCLQQYDPGSQGAYKI---AMIKK-IYKKHVCRHTYRTPTWKW--FTPKPN-W-LKEPPVFNPRSGW-TRRGFAPFKEYQIIQGREIWCCVA--ESP-E-F-CDQEC----L--MPMGIQIYANNHDPAATYK--------TQWAIEAYQSKGKVHVIDPRVGSCAMQMWDE-S---RQNHECHLWPYEKYRGEDQS--R-YNSEMTRNYCSEKMPDCDDSMFRNKCHFDKGKYDIGFPCEGE-S--ESHHDVVWANFSSFRQTLHLMQHRRMF----HTYQDRAEVPHWHM-HDKKRI-K--DH-GCFYNFPPKINN-----T-L-IHMMMLKPKPQPSPIEAQWKKDCKQYTVCEYEDIWRQSNVRSHHGWQPNFPKKYNTDSWEGDDETVFLPMWQGIFLYF-E--KVHYI-T---LFAECMVISNALVIILQDTYVYNLM---C-N---MPWGIHHRHSGFVVEMCTMTMHYKLKIYPHP-C-FTHHSNRM--AY-TW-SW--PQQQ-ARATKYCVIWPNITNCSLKWFLWAMDLGSAAIQCPKWE-ITIPQVEYSPYFSMKSRPPPAQWGYWRQSRQQTNDWFHPIWIEGPPQALYI-AQFICIDLAPGRVYQSCCPFLEMGFYYRAGQYYEHRKVGKVYDIEPMRTWHN

In [21]:
x, y = [i.strip() for i in open('input/rosalind_ba5l.txt', 'r')]
D, max_score = globalAlignment(x, y, blosum62)
algn = traceback(D, x, y, blosum62)
print max_score
print '\n'.join(algn)

18907
-EY------V--NHRSPWFMAWPHMTWIT----QRCRR-MIWRQCQGHPQIAK-YWDCPHHHNDQKARKTKVDMVICDGFPKEFLSTDGYRFLWWSGWLQFIRWNADFIR-N--TYQ-LCGK-AF--LVRACSGREALEQDQ-QYAMFQNDESQFG-IC-KNPLPVNPKDANGHRYRP--GHQSHEKTKIVMALDAWTYM--T-ARK-IDSRSF--DRAADHRKVRFCTTAPYAIGYNEFIYQNEWDRKICD-FIKYVWCINNFQKKTGRSIGFNLCKYGQAGGWGMMHIKKFQHMVMEKFTNPRKMKDIEIKLRVWQVLRRMSAGWDTMFISCYRMVKRMIRGVEDMRQPVECTGRHRWT--L-TIYIDSRNFLWDRCNYNGIDYCE---C-----GWRDYSYCMYWYHFQLNYAGFNKYMKASTSFI-MHNSINLPESIANT-------FNGQSWIRDEYADASNEPVSIPAGRYNTFHCHG-MERWHRHVMNQITVQGFQWWAACCKSREMWRWLDRWAMQNIAKVHSTSE-AQNYWINHHGAIQQCGYLGDCHTCQGWQYQSQQLWDWIHWMGAKWLWRMHLLRKLGIHVIYRMQ--PE-NL-H----VSDKMSPTE-------EL-----K-------DQESVYN-----M-D--E-----LLDVY----LLMYYCQERQVLQPQKHHPIIWFVGFIAKCNTW-----GCVRRPIFKQCQWPDSENASKG-HAE---QDER-AVAKDVRPPFIIID----QTNPEA-----C-K---GYDVPD--H----LVFQLPHDGTARHCRHTNYDQEDSYTHGTHTHLAEWVFGQYPQVTKMLVKDHK--SLKFDQWFRSLLD-D-NVQKFPLKTWCSATIAVDPHHCIPKCRIFFFVPHSWYTFSIFKICYQSMVIGRKMWQCMSKTLNNYKPDTNDYDTQKQTENRWYTPMMGSHCYQGEAWFPINNPEHGKQVACKGIGVQTHIRYQDQTI